In [2]:
import cv2
import pytesseract as tess
import numpy as np
from pdf2image import convert_from_path
from PIL import Image
import re
import pandas as pd
# help(tess)
tess.pytesseract.tesseract_cmd  = r'C:\Users\Pfactorial00110\anaconda3\Library\bin\tesseract'
#poppler_path = r'C:\Users\Pfactorial00110\Downloads\poppler-0.68.0_x86\poppler-0.68.0\bin'

In [3]:
def ocr_pdf(pdf):

    images = convert_from_path(pdf)
    img = cv2.cvtColor(np.array(images[0]), cv2.COLOR_RGB2BGR)
    content=tess.image_to_string(img)
    return content

In [4]:
images = convert_from_path('02162022.pdf')
im_list = [im1.crop((600,50,im1.size[0]-50,im1.size[1]-50)) for im1 in images]
#initial testing with saving as separate pdf might remove in future
# im_list[0].save(r'turned_pdf.pdf', save_all=True, append_images=im_list[1:])

In [46]:
full_text = {}
for i,ims in enumerate(im_list):
    con = tess.image_to_string(ims)
    full_text[i] = con

In [6]:
item_cat_sold = re.findall(r"ITEM.+",full_text[6].replace('\n','***'))[0].split('ORDER ENTRY')[0]
item_cat_sold

'ITEM CATEGORIES SOLD***Category Units Gross***CLASSIC THIN CRUST 1 13.29***COLD SUBS 1 7A9***CRUSTS 58 1,023.87***CRUSTS (modifier) 4 0.00***DRINKS 9 8.37***EXTRAS 8 84.92***HOT HOAGIES 3 25.97***MISCELLANEOUS 1 0.65***MISCELLANEOUS (modifier) 4 0.00***MODIFIERS 2 1.30***MODIFIERS (modifier) 115 0.00***PIZZA BY THE SLICE 103 697.67***SALADS 2 10.98***'

# page 7

In [7]:
def item_category_sold():
    item_cat_sold = re.findall(r"ITEM.+",full_text[6].replace('\n','***'))[0].split('ORDER ENTRY')[0]
    item_cat_sold_1 = item_cat_sold.split('***')
    item_cat_sold_1 = [i.replace(',','') for i in item_cat_sold_1]

    colnames = item_cat_sold_1[1].split(' ')
    cat_sold = pd.DataFrame()
    for i in range(len(item_cat_sold_1)):
        try:
            category = re.findall('.+?\d',item_cat_sold_1[i])[0][:-2]
            units = re.findall('\d+',item_cat_sold_1[i])[0]
            gross = re.findall('\d+[.]\d+',item_cat_sold_1[i])[0]
            # print(category,units,gross)
            cat_sold = cat_sold.append(pd.Series([category,units,gross]).T, ignore_index=True)
        except:
            pass
    cat_sold.columns = colnames
    return cat_sold


In [8]:
cat_sold = item_category_sold()
cat_sold

,Category,Units,Gross
0,CLASSIC THIN CRUST,1,13.29
1,CRUSTS,58,1023.87
2,CRUSTS (modifier),4,0.00
3,DRINKS,9,8.37
4,EXTRAS,8,84.92
5,HOT HOAGIES,3,25.97
6,MISCELLANEOUS,1,0.65
7,MISCELLANEOUS (modifier),4,0.00
8,MODIFIERS,2,1.30
9,MODIFIERS (modifier),115,0.00


# page 5

In [43]:
def sales_by_employee():
    emp_sales = re.findall(r"SALES BY EMPLOYEE.+",full_text[4].replace('\n','***'))[0].split('SALES BY ORDER TYPE')[0]
    emp_sales_1 = emp_sales.split('***')
    emp_sales_1 = [i.replace(',','') for i in emp_sales_1]

    # colnames = emp_sales_1[1].split(' ')
    sales_employee = pd.DataFrame()
    for i in range(len(emp_sales_1)):
        try:
            ename = re.findall('.+?\d',emp_sales_1[i])[0][:-2]
            count = re.findall('\d+',emp_sales_1[i])[0]
            gross = re.findall('\d+[.]\d+',emp_sales_1[i])[-2]
            net = re.findall('\d+[.]\d+',emp_sales_1[i])[-1]
            # print(ename,gross,net)
            sales_employee = sales_employee.append(pd.Series([ename,count,gross,net]).T, ignore_index=True)
        except:
            pass
    sales_employee.columns = ['ename','count','gross','net']
    return sales_employee


In [44]:
sales_employee = sales_by_employee()
sales_employee

,ename,count,gross,net
0,BOIES NATE,6,164.78,164.78
1,BOTHUN ANTHONY,6,83.12,82.18
2,CARNEY JACK,14,306.85,295.84
3,EASTLUND ANTHONY,78,669.86,669.86
4,FLAXMAN MARK,2,23.36,23.36
5,JENSEN IAN,10,231.83,216.68
6,Total,116,1479.80,1452.70


In [11]:
sales_summ = re.findall(r"SALES SUMMARY.+",full_text[0].replace('\n','***'))[0].split('TENDER SUMMARY')[0]
sales_summ_1 = sales_summ.split('***')
sales_summ_1

['SALES SUMMARY',
 'Gross Item Sales 1,874.51',
 'Delivery Charges 17 55.00',
 'Total Gross Sales 1,929.51',
 'Coupons',
 'Non-Vouchered 7 25.07',
 'Customer Credits 1 7.99',
 'Discounts 1 1.20',
 '34.26',
 'NET SALES $1,895.25',
 '']

# page 1

In [12]:
def sales_summary():
    sales_summ = re.findall(r"SALES SUMMARY.+",full_text[0].replace('\n','***'))[0].split('TENDER SUMMARY')[0]
    sales_summ_1 = sales_summ.split('***')
    sales_summ_1 = [i.replace(',','') for i in sales_summ_1]
    # colnames = sales_summ_1[1].split(' ')
    summary_sales = pd.DataFrame()
    for i in range(len(sales_summ_1)):
        try:
            type_main = re.findall('.+?\d',sales_summ_1[i])[0][:-2]
            try:
                count = re.findall('\d+\s',sales_summ_1[i])[0]
            except:
                count =  'NULL'
            net = re.findall('\d+[.]\d+',sales_summ_1[i])[0]
            # print(ename,gross,net)
            summary_sales = summary_sales.append(pd.Series([type_main,count,net]).T, ignore_index=True)
        except:
            pass
    summary_sales.columns = ['type_main','count','net']
    return summary_sales


In [13]:
summary_sales = sales_summary()
summary_sales

,type_main,count,net
0,Gross Item Sales,NULL,1874.51
1,Delivery Charges,17,55.00
2,Total Gross Sales,NULL,1929.51
3,Non-Vouchered,7,25.07
4,Customer Credits,1,7.99
5,Discounts,1,1.20
6,,NULL,34.26
7,NET SALES,NULL,1895.25


In [14]:
item_cat_sold_1 = item_cat_sold.split('***')
item_cat_sold_1[1]

'Category Units Gross'

In [15]:
colnames = item_cat_sold_1[1].split(' ')
colnames

['Category', 'Units', 'Gross']

In [16]:
category = re.findall('.+?\d',item_cat_sold_1[2])[0][:-2]
category

'CLASSIC THIN CRUST'

In [17]:
units = re.findall('\d+',item_cat_sold_1[-3])[0]
units

'103'

In [18]:
gross = re.findall('\d+[.]\d+',item_cat_sold_1[6])[0]
gross

'8.37'

# page 11

In [31]:
def sales_by_remote_source():
    remote_sales = re.findall(r"SALES BY REMOTE.+",full_text[10].replace('\n','***'))[0].split('HOUSE ACCOUNT ACTIVITY')[0]
    remote_sales_1 = remote_sales.split('***')
    remote_sales_1 = [i.replace(',','') for i in remote_sales_1]
    # colnames = sales_summ_1[1].split(' ')
    names = pd.DataFrame()
    for i in range(len(remote_sales_1)):
        try:
            type_main = re.findall('.+?\d',remote_sales_1[i])[0][:-2]
            try:
                count = re.findall('\d+\s',remote_sales_1[i])[0]
            except:
                count =  'NULL'
            Gross = re.findall('\d+[.]\d+',remote_sales_1[i])[0]
            Net = re.findall('\d+[.]\d+',remote_sales_1[i])[-1]
            # print(ename,gross,net)
            names = names.append(pd.Series([type_main,count,Gross,Net]).T, ignore_index=True)
        except:
            pass
    names.columns = ['type_main','count','Gross','Net']
    print(remote_sales)
    return names
    print(remote_sales)

In [32]:
names = sales_by_remote_source()
names

SALES BY REMOTE ORDER SOURCE****** ******Gross Net******SpeedDine 12 449.71 442.55******Total 12 $449.71 $442.55***


,type_main,count,Gross,Net
0,SpeedDine,12,449.71,442.55
1,Total,12,449.71,442.55


# page 9

In [21]:
def order_charge_summary():
    order_charges = re.findall(r"ORDER CHARGE.+",full_text[8].replace('\n','***'))[0].split('ORDER CREDIT')[0]
    order_charges_1 = order_charges.split('***')
    order_charges_1 = [i.replace(',','') for i in order_charges_1]
    # colnames = sales_summ_1[1].split(' ')
    orders = pd.DataFrame()
    for i in range(len(order_charges_1)):
        try:
            Category = re.findall('.+?\d',order_charges_1[i])[0][:-2]
            try:
                Quantity = re.findall('\d+\s',order_charges_1[i])[0]
            except:
                Quantity =  'NULL'
            Price = re.findall('\d+[.]\d+',order_charges_1[i])[0]
            # print(ename,gross,net)
            orders = orders.append(pd.Series([Category,Quantity,Price]).T, ignore_index=True)
        except:
            pass
    orders.columns = ['Category','Quantity','Price']
    return orders

In [33]:
orders = order_charge_summary()
orders

,Category,Quantity,Price
0,Delivery Charge,17,55.00
1,Gratuity,0,0.00
2,Misc,0,0.00
3,Tender Surcharge,0,0.00
4,Order Surcharge,0,0.00
5,Min Charge,0,0.00
6,,17,55.00


# page 9

In [23]:
def order_credit_summary():
    order_credit = re.findall(r"ORDER CREDIT.+",full_text[8].replace('\n','***'))[0].split('PAY-IN')[0]
    order_credit_1 = order_credit.split('***')
    order_credit_1 = [i.replace(',','') for i in order_credit_1]
    # colnames = sales_summ_1[1].split(' ')
    credits = pd.DataFrame()
    for i in range(len(order_credit_1)):
        try:
            Category = re.findall('.+?\d',order_credit_1[i])[0][:-2]
            Quantity = re.findall('\d+\s',order_credit_1[i])[0]
            Price = re.findall('\d+[.]\d+',order_credit_1[i])[0]
            # print(ename,gross,net)
            credits = credits.append(pd.Series([Category,Quantity,Price]).T, ignore_index=True)
        except:
            pass
    credits.columns = ['Category','Quantity','Price']
    return credits

In [47]:
credit = order_credit_summary()
credit

,Category,Quantity,Price
0,Coupon,7,25.07
1,Discount,1,1.20
2,Comp,0,0.00
3,Customer Credit,1,7.99
4,Order Discount,0,0.00
5,9,9,34.26
